# **Lab 2 Basim Sherief 1210207**


### **Requirement 2**

1) Complete the provided matrix addition example, following these cases:
        
        A.   kernel1: each thread produces one output matrix element
        B.   kernel2: each thread produces one output matrix row
        C.   kernel3: each thread produces one output matrix column
  Analyze the pros and cons of each of the kernels above by using nvprof with large matrix sizes to validate your posize_ts. Collect your insights in a PDF report and explain them.

2) Implement a matrix–vector multiplication kernel. Use one thread to calculate an output vector element.

Let both programs read testcases from a .txt file and prsize_t the output to another. Their pathes are to be provided as command line arguments. Sample test file and invoking command are to be attached to the e-learning page.



In [1]:
# Setup cuda environment
%pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\basim\appdata\local\temp\pip-req-build-i4rsh6yg
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.
Source files will be saved in "C:\Users\basim\AppData\Local\Temp\tmp42z12e90".


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\basim\AppData\Local\Temp\pip-req-build-i4rsh6yg'


**Generate Testcases**

In [2]:
import numpy as np

def generate_matrix_test_cases(num_tests, min_dim, max_dim, min_val, max_val, output_file):
    """
    Generate test cases for matrix addition
    Parameters:
    - num_tests: number of test cases
    - min_dim: minimum dimension (rows/cols)
    - max_dim: maximum dimension (rows/cols)
    - min_val: minimum value in matrices
    - max_val: maximum value in matrices
    - output_file: path to output file
    """
    with open(output_file, 'w') as f:
        # Write number of test cases
        f.write(f"{num_tests}\n")
        
        for _ in range(num_tests):
            # Generate random dimensions
            rows = np.random.randint(min_dim, max_dim + 1)
            cols = np.random.randint(min_dim, max_dim + 1)
            
            # Write dimensions
            f.write(f"{rows} {cols}\n")
            
            # Generate and write first matrix
            matrix1 = np.random.uniform(min_val, max_val, (rows, cols))
            for row in matrix1:
                f.write(" ".join(f"{x:.3f}" for x in row) + "\n")
            
            # Generate and write second matrix
            matrix2 = np.random.uniform(min_val, max_val, (rows, cols))
            for row in matrix2:
                f.write(" ".join(f"{x:.3f}" for x in row) + "\n")

# Set your parameters here
params = {
    'num_tests': 5,              # Number of test cases
    'min_dim': 2,               # Minimum matrix dimension
    'max_dim': 10,               # Maximum matrix dimension
    'min_val': -50000000.0,           # Minimum value in matrices
    'max_val': 5000000000.0,            # Maximum value in matrices
    'output_file': './inputfile.txt'  # Output file name
}

# Run the generator with the specified parameters
if __name__ == "__main__":
    generate_matrix_test_cases(**params)

# **CPU Only**
# Vector addition in pure C (CPU-only execution)

In [3]:
%%writefile kernel0.cu   
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <time.h>

#define MAX_ERR 1e-6

// Function to perform vector addition
void vector_add(double *out, double *a, double *b, size_t  n) {
    for (size_t  i = 0; i < n; i++) {
        out[i] = a[i] + b[i];
    }
}

int main(int argc, char* argv[]) {
    FILE *file_reading;
    int numberOfTests;
    size_t rows, cols;
    // Open the file in read mode
    file_reading = fopen(argv[1], "r");
    if (file_reading == NULL) {
        printf("Error opening file!\n");
        return 1;
    }

    // Read number of tests
    fscanf(file_reading, "%d",&numberOfTests);
for(size_t i=0;i<numberOfTests;i++){
    
    // Read matrix dimensions
    fscanf(file_reading, "%zu %zu",&rows, &cols);
    // Allocate host matrices
    double* A = (double*)malloc(sizeof(double) * rows * cols);
    double* B = (double*)malloc(sizeof(double) * rows * cols);
    double* C = (double*)malloc(sizeof(double) * rows * cols);

    if (A == NULL || B == NULL || C == NULL) {
        printf("Memory allocation failed!\n");
        fclose(file_reading);
        return 1;
    }

    // Read matrices A and B
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &A[i]);
    }
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &B[i]);
    }



    // Start timing
    clock_t start = clock();

    // Perform vector addition
    vector_add(C, A, B, rows * cols);

    // End timing
    clock_t end = clock();

    // Calculate the elapsed time in seconds
    double time_spent = (double)(end - start) / CLOCKS_PER_SEC * 1000.0;

    printf("Time elapsed: %f ms\n", time_spent);

    // Verification
    for (size_t i = 0; i < rows * cols; i++) {
        assert(fabs(C[i] - A[i] - B[i]) < MAX_ERR);
    }

    printf("Vector addition completed successfully!\n");

  
    // Write results to output file
   // Write results to output file
    FILE *file_writing;
    file_writing= fopen(argv[2], "w"); // Open file for writing
    if (file_writing == NULL) {
        perror("Error opening file");
        return 1;
    }


    // Write matrix C
    for (size_t i = 0; i < rows; i++) {
        for (size_t j = 0; j < cols; j++) {
              printf("%.3lf ", C[i * cols + j]);    
            fprintf(file_writing, "%.3lf ", C[i * cols + j]); // Write double with 2 decimal places
        }
         printf("\n");  
        fprintf(file_writing, "\n"); // New line after each row
    }
    fclose(file_writing);


    // Free allocated memory
    free(A);
    free(B);
    free(C);
}
    return 0;
}

Overwriting kernel0.cu


In [ ]:
# Compile the CUDA program
!nvcc kernel0.cu -o kernel0.exe


kernel0.cu
tmpxft_00001b88_00000000-10_kernel0.cudafe1.cpp
   Creating library kernel0.lib and object kernel0.exp
Time elapsed: 0.000000 ms
Vector addition completed successfully!
6435366487.397 1433178484.549 1510275566.968 1828845876.877 4483950868.614 4166451532.966 7074156538.042 4541242014.452 4837375348.743 8117219560.029 
6575081993.646 4886121097.452 4007272978.500 4812473309.278 5173170920.441 846818149.884 9062404273.921 6520157039.148 2726124482.217 5840637726.005 
8009372404.035 4441444876.484 5269302642.615 4512346469.820 2819923498.570 4296742355.366 2681748014.299 5074690285.724 6157783623.263 2653662371.780 
4151113376.366 9019137420.064 2907495850.631 5218992329.786 6116726160.825 3995332147.349 6218736239.915 3158080352.853 6635297222.719 4046196482.220 
3829458124.007 771164488.591 5023396724.300 3939093889.824 3665183783.752 4810321684.818 4053150843.068 5071842735.070 5408255950.289 5519999999.163 
5361094170.047 8661846465.107 1760658262.295 4708091841.541 4797400

In [15]:
# Run the executable (Windows style)
!.\kernel0.exe inputfile.txt outputfile_cpu.txt

Time elapsed: 0.000000 ms
Vector addition completed successfully!
6435366487.397 1433178484.549 1510275566.968 1828845876.877 4483950868.614 4166451532.966 7074156538.042 4541242014.452 4837375348.743 8117219560.029 
6575081993.646 4886121097.452 4007272978.500 4812473309.278 5173170920.441 846818149.884 9062404273.921 6520157039.148 2726124482.217 5840637726.005 
8009372404.035 4441444876.484 5269302642.615 4512346469.820 2819923498.570 4296742355.366 2681748014.299 5074690285.724 6157783623.263 2653662371.780 
4151113376.366 9019137420.064 2907495850.631 5218992329.786 6116726160.825 3995332147.349 6218736239.915 3158080352.853 6635297222.719 4046196482.220 
3829458124.007 771164488.591 5023396724.300 3939093889.824 3665183783.752 4810321684.818 4053150843.068 5071842735.070 5408255950.289 5519999999.163 
5361094170.047 8661846465.107 1760658262.295 4708091841.541 4797400952.902 7130418557.950 5559034200.029 6038905640.509 6143575485.546 2105827621.119 
2711240274.821 3438371894.433 

In [14]:

# For profiling with nvprof (Windows style)
!nvprof .\kernel0.exe inputfile.txt outputfile_cpu.txt

# kernel1: each thread produces one output matrix element


In [5]:
%%writefile kernel1.cu   
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <fstream>
#include <iostream>
#include <sstream>
#define MAX_ERR 1e-6
__global__ void matrixAddKernel1(double* C, double* A, double* B, size_t rows, size_t cols) {
    size_t row = blockIdx.y * blockDim.y + threadIdx.y;
    size_t col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < rows && col < cols) {
        size_t idx = row * cols + col;
        C[idx] = A[idx] + B[idx];
    }
}

cudaError_t addMatricesWithCuda(double* C, double* A, double* B, size_t rows, size_t cols) {
    double* dev_A = nullptr;
    double* dev_B = nullptr;
    double* dev_C = nullptr;
    cudaError_t cudaStatus;

    // Allocate GPU buffers
    size_t size = rows * cols * sizeof(double);  // Changed from size_t to double
    
    cudaStatus = cudaMalloc((void**)&dev_C, size);
    
    cudaStatus = cudaMalloc((void**)&dev_A, size);

    cudaStatus = cudaMalloc((void**)&dev_B, size);

    // Copy input matrices from host memory to GPU buffers
    cudaStatus = cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);    
    cudaStatus = cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(16, 16); // i just did what Ta and cuda said the best to use
    dim3 numBlocks((cols + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (rows + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrixAddKernel1<<<numBlocks, threadsPerBlock>>>(dev_C, dev_A, dev_B, rows, cols);

    // Copy output matrix from GPU buffer to host memory
    cudaStatus = cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    return cudaStatus;
}

int main(int argc, char* argv[]) {

    FILE *file_reading;
    int numberOfTests;
    size_t  rows, cols;
    // Open the file in read mode
    file_reading = fopen(argv[1], "r");
    if (file_reading == NULL) {
        printf("Error opening file!\n");
        return 1;
    }
    // Read number of tests
    fscanf(file_reading, "%d",&numberOfTests);
for(size_t i=0;i<numberOfTests;i++){
    
    // Read matrix dimensions
    fscanf(file_reading, "%zu %zu", &rows, &cols);

    // Allocate host matrices
    double* A = (double*)malloc(sizeof(double) * rows * cols);  // Changed from size_t to double
    double* B = (double*)malloc(sizeof(double) * rows * cols);
    double* C = (double*)malloc(sizeof(double) * rows * cols);

    // Read matrices A and B
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &A[i]);
    }
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &B[i]);
    }
    
 

    // Add matrices using CUDA
    cudaError_t cudaStatus = addMatricesWithCuda(C, A, B, rows, cols);

    // Verification
    for (size_t i = 0; i < rows * cols; i++) {
        assert(fabs(C[i] - A[i] - B[i]) < MAX_ERR);
    }

    printf("Vector addition completed successfully!\n");

    // Write results to output file
    FILE *file_writing;
    file_writing= fopen(argv[2], "w"); // Open file for writing
    if (file_writing == NULL) {
        perror("Error opening file");
        return 1;
    }


    // Write matrix C
    for (size_t i = 0; i < rows; i++) {
        for (size_t j = 0; j < cols; j++) {
              printf("%.3f ", C[i * cols + j]);    
            fprintf(file_writing, "%.3f ", C[i * cols + j]); // Write double with 2 decimal places
        }
         printf("\n");  
        fprintf(file_writing, "\n"); // New line after each row
    }
    fclose(file_writing);


    // Cleanup
    free(A);
    free(B);
    free(C);
}
    return 0;
}

Overwriting kernel1.cu


In [ ]:
# Compile the CUDA program
!nvcc kernel1.cu -o kernel1.exe

# Run the executable (Windows style)
!.\kernel1.exe inputfile.txt outputfile_cpu.txt

# perfomance for cuda is depricacted but can be used in kaggle notebook
# !nvprof .\kernel1.exe inputfile.txt outputfile_cpu.txt



In [22]:
# For profiling with nvprof (Windows style) the new one  for system-level profiling
# !nsys profile --stats=true .\kernel1.exe inputfile.txt outputfile_cpu.txt
# , for kernel details using Nsight Compute:
!ncu --set full .\kernel1.exe inputfile.txt outputfile_cpu.txt

==PROF== Connected to process 12764 (E:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Lab_2_Notebook_Solution\kernel1.exe)
==PROF== Profiling "matrixAddKernel1" - 0: 0%....50%....100% - 39 passes
==PROF== Profiling "matrixAddKernel1" - 1: 0%....50%....100% - 39 passes
==PROF== Profiling "matrixAddKernel1" - 2: 0%....50%....100% - 39 passes
==PROF== Profiling "matrixAddKernel1" - 3: 0%....50%....100% - 39 passes
==PROF== Profiling "matrixAddKernel1" - 4: 0%....50%....100% - 39 passes
Vector addition completed successfully!
6435366487.397 1433178484.549 1510275566.968 1828845876.877 4483950868.614 4166451532.966 7074156538.042 4541242014.452 4837375348.743 8117219560.029 
6575081993.646 4886121097.452 4007272978.500 4812473309.278 5173170920.441 846818149.884 9062404273.921 6520157039.148 2726124482.217 5840637726.005 
8009372404.035 4441444876.484 5269302642.615 4512346469.820 2819923498.570 4296742355.366 2681748014.299 5074690285.724 6157783623.263 2653662371.

# kernel2: each thread produces one output matrix row


In [7]:
%%writefile kernel2.cu   
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <fstream>
#include <iostream>
#include <sstream>
#define MAX_ERR 1e-6
__global__ void matrixAddKernel1(double* C, double* A, double* B, size_t rows, size_t cols) {
    size_t row = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < rows) {
        for(size_t col =0 ; col < cols; col++){
              size_t idx = row * cols + col;
            C[idx] = A[idx] + B[idx];
        }
    }
}

cudaError_t addMatricesWithCuda(double* C, double* A, double* B, size_t rows, size_t cols) {
    double* dev_A = nullptr;
    double* dev_B = nullptr;
    double* dev_C = nullptr;
    cudaError_t cudaStatus;

    // Allocate GPU buffers
    size_t size = rows * cols * sizeof(double);  // Changed from size_t to double
    
    cudaStatus = cudaMalloc((void**)&dev_C, size);
    
    cudaStatus = cudaMalloc((void**)&dev_A, size);

    cudaStatus = cudaMalloc((void**)&dev_B, size);

    // Copy input matrices from host memory to GPU buffers
    cudaStatus = cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);    
    cudaStatus = cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(16, 16); // i just did what Ta and cuda said the best to use
    dim3 numBlocks((cols + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (rows + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrixAddKernel1<<<numBlocks, threadsPerBlock>>>(dev_C, dev_A, dev_B, rows, cols);

    // Copy output matrix from GPU buffer to host memory
    cudaStatus = cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    return cudaStatus;
}

int main(int argc, char* argv[]) {

    FILE *file_reading;
    int numberOfTests;
    size_t  rows, cols;
    // Open the file in read mode
    file_reading = fopen(argv[1], "r");
    if (file_reading == NULL) {
        printf("Error opening file!\n");
        return 1;
    }
    // Read number of tests
    fscanf(file_reading, "%d",&numberOfTests);
for(size_t i=0;i<numberOfTests;i++){
    
    // Read matrix dimensions
    fscanf(file_reading, "%zu %zu", &rows, &cols);

    // Allocate host matrices
    double* A = (double*)malloc(sizeof(double) * rows * cols);  // Changed from size_t to double
    double* B = (double*)malloc(sizeof(double) * rows * cols);
    double* C = (double*)malloc(sizeof(double) * rows * cols);

    // Read matrices A and B
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &A[i]);
    }
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &B[i]);
    }
    
 

    // Add matrices using CUDA
    cudaError_t cudaStatus = addMatricesWithCuda(C, A, B, rows, cols);

    // Verification
    for (size_t i = 0; i < rows * cols; i++) {
        assert(fabs(C[i] - A[i] - B[i]) < MAX_ERR);
    }

    printf("Vector addition completed successfully!\n");

    // Write results to output file
    FILE *file_writing;
    file_writing= fopen(argv[2], "w"); // Open file for writing
    if (file_writing == NULL) {
        perror("Error opening file");
        return 1;
    }


    // Write matrix C
    for (size_t i = 0; i < rows; i++) {
        for (size_t j = 0; j < cols; j++) {
              printf("%.3f ", C[i * cols + j]);    
            fprintf(file_writing, "%.3f ", C[i * cols + j]); // Write double with 2 decimal places
        }
         printf("\n");  
        fprintf(file_writing, "\n"); // New line after each row
    }
    fclose(file_writing);


    // Cleanup
    free(A);
    free(B);
    free(C);
}
    return 0;
}

Overwriting kernel2.cu


In [8]:
!nvcc kernel2.cu -o kernel2
!nvprof ./kernel2 inputfile.txt outputfile_kernel2.txt

kernel2.cu
tmpxft_00003b94_00000000-10_kernel2.cudafe1.cpp
   Creating library kernel2.lib and object kernel2.exp


# kernel3: each thread produces one output matrix col


In [9]:
%%writefile kernel3.cu   
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <fstream>
#include <iostream>
#include <sstream>
#define MAX_ERR 1e-6
__global__ void matrixAddKernel1(double* C, double* A, double* B, size_t rows, size_t cols) {
    size_t col = blockIdx.x * blockDim.x + threadIdx.x;
    if ( col < cols) {
        for(size_t row = 0; row< rows ; row++){
            size_t idx = row * cols + col;
            C[idx] = A[idx] + B[idx];
        }
   
    }
}

cudaError_t addMatricesWithCuda(double* C, double* A, double* B, size_t rows, size_t cols) {
    double* dev_A = nullptr;
    double* dev_B = nullptr;
    double* dev_C = nullptr;
    cudaError_t cudaStatus;

    // Allocate GPU buffers
    size_t size = rows * cols * sizeof(double);  // Changed from size_t to double
    
    cudaStatus = cudaMalloc((void**)&dev_C, size);
    
    cudaStatus = cudaMalloc((void**)&dev_A, size);

    cudaStatus = cudaMalloc((void**)&dev_B, size);

    // Copy input matrices from host memory to GPU buffers
    cudaStatus = cudaMemcpy(dev_A, A, size, cudaMemcpyHostToDevice);    
    cudaStatus = cudaMemcpy(dev_B, B, size, cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(16, 16); // i just did what Ta and cuda said the best to use
    dim3 numBlocks((cols + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (rows + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrixAddKernel1<<<numBlocks, threadsPerBlock>>>(dev_C, dev_A, dev_B, rows, cols);

    // Copy output matrix from GPU buffer to host memory
    cudaStatus = cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);

    return cudaStatus;
}

int main(int argc, char* argv[]) {

    FILE *file_reading;
    int numberOfTests;
    size_t  rows, cols;
    // Open the file in read mode
    file_reading = fopen(argv[1], "r");
    if (file_reading == NULL) {
        printf("Error opening file!\n");
        return 1;
    }
    // Read number of tests
    fscanf(file_reading, "%d",&numberOfTests);
for(size_t i=0;i<numberOfTests;i++){
    
    // Read matrix dimensions
    fscanf(file_reading, "%zu %zu", &rows, &cols);

    // Allocate host matrices
    double* A = (double*)malloc(sizeof(double) * rows * cols);  // Changed from size_t to double
    double* B = (double*)malloc(sizeof(double) * rows * cols);
    double* C = (double*)malloc(sizeof(double) * rows * cols);

    // Read matrices A and B
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &A[i]);
    }
    for (size_t i = 0; i < rows * cols; i++) {
        fscanf(file_reading, "%lf", &B[i]);
    }
    
 

    // Add matrices using CUDA
    cudaError_t cudaStatus = addMatricesWithCuda(C, A, B, rows, cols);

    // Verification
    for (size_t i = 0; i < rows * cols; i++) {
        assert(fabs(C[i] - A[i] - B[i]) < MAX_ERR);
    }

    printf("Vector addition completed successfully!\n");

    // Write results to output file
    FILE *file_writing;
    file_writing= fopen(argv[2], "w"); // Open file for writing
    if (file_writing == NULL) {
        perror("Error opening file");
        return 1;
    }


    // Write matrix C
    for (size_t i = 0; i < rows; i++) {
        for (size_t j = 0; j < cols; j++) {
            printf("%.3f ", C[i * cols + j]);    
            fprintf(file_writing, "%.3f ", C[i * cols + j]); // Write double with 2 decimal places
        }
        printf("\n");  
        fprintf(file_writing, "\n"); // New line after each row
    }
    fclose(file_writing);


    // Cleanup
    free(A);
    free(B);
    free(C);
}
    return 0;
}

Overwriting kernel3.cu


In [10]:
!nvcc kernel3.cu -o kernel3
!nvprof ./kernel3 inputfile.txt outputfile_kernel3.txt

kernel3.cu
tmpxft_00001f04_00000000-10_kernel3.cudafe1.cpp
   Creating library kernel3.lib and object kernel3.exp
